![Two data scientists working on a dashboard.](hr-image-small.png)

Um problema comum ao criar modelos para gerar valor de negócio a partir de dados é que os conjuntos de dados podem ser tão grandes que o modelo pode levar dias para gerar previsões. Garantir que seu conjunto de dados esteja armazenado da forma mais eficiente possível é crucial para permitir que esses modelos rodem em um prazo mais razoável, sem precisar reduzir o tamanho do conjunto de dados.

Você foi contratado por uma grande empresa de treinamento em ciência de dados online chamada Training Data Ltd. para limpar um dos maiores conjuntos de dados de clientes. Esse conjunto de dados será eventualmente usado para prever se seus estudantes estão procurando um novo emprego ou não, informação que será então utilizada para direcioná-los a recrutadores em potencial.

Você recebeu acesso ao customer_train.csv, que é um subconjunto de todo o conjunto de dados de clientes, para que possa criar uma prova de conceito de uma solução de armazenamento muito mais eficiente. O conjunto de dados contém informações anonimizadas dos estudantes, e se eles estavam ou não procurando um novo emprego durante o treinamento:


| Column                   | Description                                                                 |
|---------------------------|------------------------------------------------------------------------------|
| student_id               | Um ID único para cada estudante.                                            |
| city                     | Um código para a cidade em que o estudante mora.                            |
| city_development_index   | Um índice de desenvolvimento escalonado para a cidade.                      |
| gender                   | O gênero do estudante.                                                      |
| relevant_experience      | Um indicador da experiência de trabalho relevante do estudante.             |
| enrolled_university      | O tipo de curso universitário em que está matriculado (se houver).          |
| education_level          | O nível de escolaridade do estudante.                                       |
| major_discipline         | A área de formação educacional do estudante.                                |
| experience               | A experiência total de trabalho do estudante (em anos).                     |
| company_size             | O número de funcionários na empresa atual do estudante.                     |
| company_type             | O tipo de empresa que emprega o estudante.                                  |
| last_new_job             | O número de anos entre o emprego atual e o anterior do estudante.           |
| training_hours           | O número de horas de treinamento concluídas.                                |
| job_change               | Um indicador de se o estudante está procurando um novo emprego (1) ou não (0). |


In [1]:
# Bibliotecas necessárias
import pandas as pd

# Carregando o dataset
ds_jobs = pd.read_csv("customer_train.csv")

# Verificando se carregamos corretamente e como é o nosso dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


1) Análise exploratória dos dados

   A primeira coisa que eu faço é entender o que eu tenho na mão:
    - Qual o tamanho do DataFrame?
    - Quais são os tipos de dados (dtypes) de cada coluna?
    - Quantos valores únicos existem em cada coluna? (isso me ajuda a decidir se é nominal, ordinal, booleana, etc.)
    - E dou uma olhada rápida nos valores mais comuns nas colunas categóricas para já planejar o que e como vou converter.

In [2]:
# 1) Análise exploratória dos dados ----------------------------------------------------------
# Visão geral
print("Shape:", ds_jobs.shape)
ds_jobs.info(memory_usage="deep")

# Cardinalidade por coluna (rápido para mapear dtypes alvo)
print("\nCardinalidade por coluna:")
print(ds_jobs.nunique(dropna=True).sort_values())

# Amostra de valores por coluna categórica (limitando top 10 pra n ficar muito longo)
cat_like = ds_jobs.select_dtypes(include=["object"]).columns.tolist()
print("\nAmostra de valores por coluna categórica (top 10):")
for c in cat_like:
    print(f"\n--- {c} ---")
    print(ds_jobs[c].value_counts(dropna=False).head(10))

Shape: (19158, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int6

2) Converter integers, floats e categorias não ordenadas
Objetivo: aplicar int32, float16, category, bool conforme os dados permitem.

Notas rápidas

Uso .astype() direto onde não há valores ausentes (NaN) problemáticos.

Para booleana de string (relevant_experience), uso .map() ⇒ True/False e depois .astype(bool).

Se alguma coluna tiver NaN e você preferir manter nulos em booleano, troque bool por o tipo pandas 'boolean' (nullable). Como o projeto pediu bool, mantenho bool.

In [3]:
# 2) Conversões base ----------------------------------------------------------
#fazendo uma cópia para transformar
ds_jobs_transformed = ds_jobs.copy()

# Numéricas diretas
int32_cols = ["student_id", "training_hours"]
for c in int32_cols:
    ds_jobs_transformed[c] = ds_jobs_transformed[c].astype("int32")

float16_cols = ["city_development_index"]
for c in float16_cols:
    ds_jobs_transformed[c] = ds_jobs_transformed[c].astype("float16")

# Bool (duas categorias)
# job_change já é 0/1 -> booleano direto
ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].astype(bool)

# relevant_experience -> map para bool
# obs: se houver valores ausentes e você quiser 'boolean' (nullable), troque para .astype("boolean")
ds_jobs_transformed["relevant_experience"] = (
    ds_jobs_transformed["relevant_experience"]
      .map({"No relevant experience": False, "Has relevant experience": True})
      .astype(bool)
)

# Nominal -> category
nominal_cols = ["city", "gender", "enrolled_university", "major_discipline", "company_type"]
for c in nominal_cols:
    ds_jobs_transformed[c] = ds_jobs_transformed[c].astype("category")
#Para conferir se todas as conversões aconteceram corretamente
print("Conversões feitas na base")

Conversões feitas na base


3) Converter categorias ordenadas

Objetivo: criar CategoricalDtype ordenado e aplicar com .astype(ordered_dtype).

Notas rápidas

Mantive suas listas (das mensagens anteriores) para experience, company_size e alinhei o natural em education_level e last_new_job.

Ordenação ascendente = do menor para o maior.

In [4]:
# 3) Categorias ordenadas -----------------------------------------------------
# enrolled_university — "nenhum curso" < "meio período" < "integral"
enrolled_order = ["no_enrollment", "Part time course", "Full time course"]
ds_jobs_transformed["enrolled_university"] = (
    ds_jobs_transformed["enrolled_university"]
      .astype("category")
      .cat.set_categories(enrolled_order, ordered=True)
      .cat.remove_unused_categories()
)

# education_level — da base para o topo acadêmico
edu_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
ds_jobs_transformed["education_level"] = (
    ds_jobs_transformed["education_level"]
      .astype("category")
      .cat.set_categories(edu_order, ordered=True)
      .cat.remove_unused_categories()
)

# experience — ordem natural do júnior ao sêniorzão (lista explícita)
exp_order = [
    '<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'
]
ds_jobs_transformed["experience"] = (
    ds_jobs_transformed["experience"]
      .astype("category")
      .cat.set_categories(exp_order, ordered=True)
      .cat.remove_unused_categories()
)

# company_size — do menor porte ao enterprise
company_size_order = [
    '<10', '10-49', '50-99', '100-499', '500-999',
    '1000-4999', '5000-9999', '10000+'
]
ds_jobs_transformed["company_size"] = (
    ds_jobs_transformed["company_size"]
      .astype("category")
      .cat.set_categories(company_size_order, ordered=True)
      .cat.remove_unused_categories()
)

# last_new_job — "nunca mudei" é o menor, ">4" é o maior
last_new_job_order = ['never', '1', '2', '3', '4', '>4']
ds_jobs_transformed["last_new_job"] = (
    ds_jobs_transformed["last_new_job"]
      .astype("category")
      .cat.set_categories(last_new_job_order, ordered=True)
      .cat.remove_unused_categories()
)

# checagem rápida
for c in [
    "enrolled_university",
    "education_level",
    "experience",
    "company_size",
    "last_new_job"
]:
    assert ds_jobs_transformed[c].cat.ordered, f"{c} não ficou como ordered=True"

#Para conferir se todas as conversões aconteceram corretamente
print("Categorias ordenadas e conversões feitas")

Categorias ordenadas e conversões feitas


4) Filtrando com base nas categorias já ordenandas

Requisito: manter apenas alunos com ≥ 10 anos de experiência e empresa com ≥ 1000 empregados.

Notas rápidas:

- Com CategoricalDtype(ordered=True), posso filtrar por comparação de string (>= '10'), porque a ordem está definida nas categorias.

- Para company_size, uso o primeiro degrau “enterprise”: '1000-4999'.

In [6]:
# 4) Filtrando com base nas categorias já ordenandas -----------------------------------------------------
print("Shape pré-filtro:", ds_jobs_transformed.shape)

filtro = (
    (ds_jobs_transformed["experience"] >= "10") &
    (ds_jobs_transformed["company_size"] >= "1000-4999")
)
ds_jobs_transformed_filtro = ds_jobs_transformed.loc[filtro].reset_index(drop=True)

print("Shape pós-filtro:", ds_jobs_transformed_filtro.shape)

Shape pré-filtro: (19158, 14)
Shape pós-filtro: (2201, 14)


5. Checagem final: dtypes + memória (antes vs. depois)

Por quê: validar que cada coluna bate o requisito e que a memória caiu mesmo de forma perceptível.

In [7]:
# 5) Checagem final: dtypes + memória (antes vs. depois) -----------------------------------------------------
# Função de checagem simples
def check_dtypes(df):
    return pd.Series({c: str(df[c].dtype) for c in df.columns})

print("\nDtypes originais:")
print(check_dtypes(ds_jobs))

print("\nDtypes transformados:")
print(check_dtypes(ds_jobs_transformed))

print("\n.info() original:")
ds_jobs.info(memory_usage="deep")

print("\n.info() transformado:")
ds_jobs_transformed.info(memory_usage="deep")

# Memória (bytes) e economia %
before = ds_jobs.memory_usage(deep=True).sum()
after  = ds_jobs_transformed.memory_usage(deep=True).sum()
saved  = before - after
pct    = (saved / before) * 100 if before else 0.0

print(f"\nMemória antes : {before:,.0f} bytes")
print(f"Memória depois: {after:,.0f} bytes")
print(f"Economia      : {saved:,.0f} bytes ({pct:.2f}%)")


Dtypes originais:
student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                float64
dtype: object

Dtypes transformados:
student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change   